## Pokemon 3: Return of the Kingler

#### Hayden Burger, Corinne Desroches, David Lee

(Tutorial requirements):

Accurately summarizes original presentation and incorporates feedback.  Effectively presents entire data analysis workflow in written form, with appropriate references.  Serves as tutorial to enable other users to understand and repeat analysis. 

Makes effective use of the GitLab platform to organize code, provide data downloading instructions, ensuring project can be replicated by other users, and includes overall aesthetics of Readme document. 


To do:

-Brief intro

-Discuss API

-Discuss python scripts

-Discuss running script/ Hamming/ shell command

-Discuss data format

-Discuss data imaging/ streamlit

### Data Gathering

### More Stuff

Talk about structure of py scripts and what's happening


The script `pokemon_script.py` imports `pokemon_module.py` as pk in order to utilize its functions. It takes as input an index number and the names of the members of a pokemon team (up to 6). In order to access these inputs, it calls the module `sys`. Each argument when called from a command line (or batch script) becomes part of the variable sys.argv. The handling, shown below, will take the inputs if they exist, and otherwise will use a default team and set the index to 1.

```python
#Arguments: poke_index and team members
if len(sys.argv) >=2:
    poke_index = int(sys.argv[1])
    player_team = []
    for i in range(2,len(sys.argv)):
        player_team.append(sys.argv[i])
else:
    #Not enough inputs
    player_team = ['gyarados','zapdos','moltres','articuno','mew','mewtwo']
    poke_index = 1
```

After that it defines lists for each of the elite four teams. We then use the pokemon_module function `create_pokemon_objects` to create teams of pokemon classes. This turns a list of pokemon names into a dictionary of pokemon class objects, indexed by their name. This can be called with `pk.create_pokemon_objects({pokemon list})`. The player team and each team in the elite four is converted to a dictionary, and the elite four dictionaries are saved in a list called `elite`.

The battle is then run using the code below, again referencing a pokemon_module function. `run_elite` will run pokemon battles for every member of the player team against every member of an opponent team, for each of the opponent teams. It is designed to take a list of exactly four teams for the elite four, and will print an error for a different team construction.

```python
results = pk.run_elite(playerteam,elite,verbose=False)
```

Results are a tuple in the form of (result, time, winner, winner_list). The `result` is a 1 or 0 for whether the player team wins or loses, respectively. `Time` is the amount of time the battle takes, assuming each round is six seconds. `Winner` returns the winning team, whether that is the player ("Champion") or a member of the Elite Four. `Winner_list` is a list of battle outcomes in order, so for every 1v1 pairing it will have either the name of the winning pokemon or "Draw". 

The results are written to an excel file, with each player team getting its own excel sheet. 

```python
columns = ['Result','Time','Winner','Winner List']

#define names for excel file
sheet_name = f"team{poke_index}"
file_name = "Output_data_files/elite_results_test.xlsx"

#add results to sheet
write_results_append(columns,results,sheet_name,file_name)
```

In order to write the results to an excel sheet we created the function `write_results_append`. This will take as input column names, results, and the sheet and file name. The function requires the `openpyxl` module. If the excel file exists it will load the file. Otherwise, it will create the file. For each sheet name it will either directly add results to that sheet, or create the sheet and add the column names before doing so.

### How to Run

#Paste sh script and talk about what the pieces mean
- Hamming

`salloc`

`sbatch pokemon_battle.sh`

`pokemon_script.py` runs a single team only once. To run multiple teams, the script is called from `run_script.py`. This selects each team from a row of `random_teams.csv` and feeds each one as an imput to `pokemon_script.py`. Below is code used to generate the random_teams.csv file. This was run in a jupyter notebook where the pokemon_module was imported.

First, a dictionary of all pokemon was created using the create_pokemon_dict function.
```python
pokemon_dict = pk.create_pokemon_dict()
```
The two functions below were used to randomly create teams and write them to a csv. `random_team_names` creates a list of n random pokemon names from the dictionary keys. `save_teams` will call the other function to create teams of six, and write each to a line in a csv. (To run it, first run `import csv` and `import numpy as np`).

In [ ]:
def random_team_names(n):
    '''Generates n pokemon names'''
    team = []
    for i in range(n):
        pokemon_name = np.random.choice(list(pokemon_dict.keys()))
        team.append(pokemon_name)
    return team

def save_teams(n,filename):
    '''writes n random teams to csv file'''
    with open(filename,'w',newline='') as file:
        writer = csv.writer(file)
        for i in range(n):
            team = random_team_names(6) 
            writer.writerow(team)
    return 
           

The code below creates a set of 4 random teams and writes them to `random_teams.csv`. `np.random.seed` was used to set the seed for repeatability. Any number in place of four can be used to write that many teams.

```python
#create a csv of 4 random teams
np.random.seed(10)
save_teams(4,'Input_data_files/random_teams.csv')
```

This file of teams is used in `run_script`. It's loaded as a pandas dataframe using `pandas.read_csv`. (`header=None` is used to indicate that there are no column names in the csv. The first line of the csv is a team).

```python
#import libraries
import pandas as pd
import os
#get teams from csv
teams = pd.read_csv("Input_data_files/random_teams.csv",header=None)
teamnumbers = len(teams)
```

For each random team, the code will create a command which runs `pokemon_script.py` with the team index number and the names of the pokemon in the team. An f string is used to add the index to the command string, and then the team names are concatenated to the end of the command using `+`. The `os` method `system()` will run the script as if from a command line.

```python
#feed teams to pokemon script (runs battlse and writes results to elite_resutls_test.xlsx)
for i in range(teamnumbers):
    
    command = f"python pokemon_script.py {i+1}"
    for value in teams.loc[i].values:
        command = command + " " + value
    
    os.system(command)
```


This entire setup so far will only run a single battle per team. This is because the intent is to run multiple battles at once using slurm on the NPS High Performance Computing system, Hamming. To do this required a batch script, `pokemon_battle.sh`. The entire script contents are below.

```bash
#!/bin/sh

#SBATCH --job-name HEB_pokemonbattle
#SBATCH --array=1-3
#SBATCH --output=Output_data_files/out_battle_%a.txt

. /etc/profile
module load lang/python
## load the comp3 virtual environment
source /smallwork/$USER/comp3/bin/activate

python -m pip install --quiet openpyxl
python run_script.py ${SLURM_ARRAY_TASK_ID} 
```

The script creates a job named HEB_pokemonbattle, divided into `3` tasks, and sends any output to out_battle_{task number}.txt. It loads python, and activates the comp3 virtual environment that was created on Hamming in this course. It ensures the environment has openpxyl installed, before running run_script.py. The `--quiet` in the pip install command mutes the output from the installation.